# Complex Classification Example

We showcase our data valuation and variable selection analyses on a dataset with a mix of 14 categorical and continuous explanatory variables, namely the UCI US census income dataset (https://archive.ics.uci.edu/ml/datasets/census+income). 

The aim here is to show that our analyses work beyond a few explanatory variables and adequately cope with categorical data.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import kxy

## The Data

The goal is predict whether a US resident's income exceeds $50k/year based on information provided in the US census questionnaire.

In [2]:
# Load the data using the kxy_datasets package (pip install kxy_datasets)
from kxy_datasets.uci_classifications import Adult
df = Adult().df
df.kxy.describe()

         
-----------
Column: Age
-----------
Type:      Categorical
Frequency:  2.76%, Label: 36
Frequency:  2.74%, Label: 35
Frequency:  2.73%, Label: 33
Frequency:  2.72%, Label: 23
Frequency:  2.71%, Label: 31
Frequency:  2.67%, Label: 34
Frequency:  2.62%, Label: 28
Frequency:  2.62%, Label: 37
Frequency:  2.62%, Label: 30
Frequency:  2.59%, Label: 38
Frequency:  2.57%, Label: 32
Frequency:  2.53%, Label: 41
Frequency:  2.52%, Label: 27
Frequency:  2.50%, Label: 29
Frequency:  2.47%, Label: 24
Frequency:  2.47%, Label: 39
Frequency:  2.45%, Label: 25
Frequency:  2.43%, Label: 40
Frequency:  2.41%, Label: 22
Frequency:  2.39%, Label: 42
Frequency:  2.36%, Label: 26
Frequency:  2.28%, Label: 20
Frequency:  2.26%, Label: 43
Frequency:  2.25%, Label: 46
Frequency:  2.24%, Label: 21
Frequency:  2.24%, Label: 45
Frequency:  2.21%, Label: 47
Frequency:  2.18%, Label: 44
Frequency:  2.16%, Label: 19
Frequency:  1.80%, Label: 51
Frequency:  1.77%, Label: 50
Frequency:  1.76%, Label: 18
Fre

## Data Valuation

About, 76% of people surveyed earned less than $50,000 annually. Thus, the baseline strategy consisting of always predicting <=50K would give us a 76% accuracy. 

Thanks to the explanatory variables, we may achieve a higher accuracy than 76%. To determine just how much higher the accuracy can get, we run our data valuation analysis. 

In [3]:
df.kxy.data_valuation('Income', problem_type='classification')

[====================================================================================================] 100% ETA: 0s   


,Achievable R-Squared,Achievable Log-Likelihood Per Sample,Achievable Accuracy
0,0.67,-1.11e-16,1.00


As it turns out demographic data can provide a substantial accuracy boost over the naive strategy.

## Variable Selection

Let us now trace which variables this boost may come from.

In [4]:
df.kxy.variable_selection('Income', problem_type='classification')

[====================================================================================================] 100% ETA: 0s   


,Variable,Running Achievable R-Squared,Running Achievable Accuracy
Selection Order,,,
0,No Variable,0.00,0.76
1,Relationship,0.21,0.89
2,Capital Gain,0.31,0.91
3,Education,0.37,0.92
4,Age,0.40,0.93
5,Hours Per Week,0.49,0.96
6,Occupation,0.57,0.98
7,Workclass,0.60,0.98
8,Race,0.61,0.99


The second row (`Relationship`) corresponds to the most important factor. When used in isolation in a classification model, an accuracy of up to 89% can be achieved. This is 13% above the accuracy that can be achieved by the naive strategy consisting of always predicting the most frequent outcome, namely `<=50K`.
 
The third row (`Capital Gain`) corresponds to the factor that complements the `Relationship` factor the most. It can bring about an additional 2% classification accuracy.

More generally, the $(i+1)$-th row corresponds to the factor that complements the first $i$ factors selected the most. 

### Comparison with alternative (model-based) factor importance analyses
The `kxy` package is the **only** Python package capable of *ex-ante* and *model-free* factor importance analysis. 

Other tools adopt a model-based approach consisting of quantifying the extent to which a trained model *relies on* each input, as opposed to the extent to which inputs are intrinsically useful for solving the problem at hand. Thus, their analyses may vary depending on the model used and as such they are subjective.

This dataset was previously studied using the tree-based SHAP method in this [notebook](https://slundberg.github.io/shap/notebooks/tree_explainer/Census%20income%20classification%20with%20LightGBM.html).

Shapley value based studies are inherently model-based as they explain the predictions made by a specific model. Additionally, they are intrinsically unrelated to accuracy or any other performance metric. Indeed, the approach merely explains model predictions (of any model, although it scales better on tree-based models), with no regard on how well said model performs.

Our approach on the other hand quantifies the importance of each variable for effectively solving the problem at hand, in a model-free fashion, and before training any predictive model. When a trained model performs optimally, Shapley values will coincide with our model-free intrinsic variable importance. However, variable selection is typically needed to help find the best model, not after it is found.